# Introduction

This tutorial is about linear transforms - a basic building block of neural networks, including deep learning models.

# Virtual environments and syncing repositories

Before you proceed onwards, remember to activate you virtual environments so you can use the software you installed last week as well as run the notebooks in interactive mode, not through the github.com website.

## Virtual environments

To activate the virtual environment:
   * If you were in last week's Tuesday or Wednesday group type `activate_mlp` or `source ~/mlpractical/venv/bin/activate`
   * If you were in the Monday group:
      + and if you have chosen the **comfy** way type: `workon mlpractical`
      + and if you have chosen the **generic** way, `source` your virutal environment using `source` and specyfing the path to the activate script (you need to localise it yourself, there were not any general recommendations w.r.t dir structure and people have installed it in different places, usually somewhere in the home directories. If you cannot easily find it by yourself, use something like: `find . -iname activate` ):

## On Synchronising repositories

Enter the git mlp repository you set up last week (i.e. `~/mlpractical/repo-mlp`) and once you sync the repository (in one of the two below ways), start the notebook session by typing:

```
ipython notebook
```

### Default way

To avoid potential conflicts between the changes you have made since last week and our additions, we recommend `stash` your changes and `pull` the new code from the mlpractical repository by typing:

1. `git stash save "Lab1 work"`
2. `git pull`

Then, if you need to, you can always (temporaily) restore a desired state of the repository.

### For advanced github users

It is OK if you want to keep your changes and merge the new code with whatever you already have, but you need to know what you are doing and how to resolve conflicts.
 

# Single Layer Models

***
### Note on storing matrices in computer memory

Suppose you want to store the following matrix in memory: $\left[ \begin{array}{ccc}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9 \end{array} \right]$ 

If you allocate the memory at once for the whole matrix, then the above matrix would be organised as a vector in one of two possible forms:

* Row-wise layout where the order would look like: $\left [ \begin{array}{ccccccccc}
1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \end{array} \right ]$
* Column-wise layout where the order would look like: $\left [ \begin{array}{ccccccccc}
1 & 4 & 7 & 2 & 5 & 8 & 3 & 6 & 9 \end{array} \right ]$

Although `numpy` can easily handle both formats (possibly with some computational overhead), in our code we will stick with the more modern (and default) `C`-like approach and use the row-wise format (in contrast to Fortran that used a column-wise approach). 

This means, that in this tutorial:
* vectors are kept row-wise $\mathbf{x} = (x_1, x_1, \ldots, x_D) $ (rather than $\mathbf{x} = (x_1, x_1, \ldots, x_D)^T$)
* similarly, in case of matrices we will stick to: $\left[ \begin{array}{cccc}
x_{11} & x_{12} & \ldots & x_{1D} \\
x_{21} & x_{22} & \ldots & x_{2D} \\
x_{31} & x_{32} & \ldots & x_{3D} \\ \end{array} \right]$ and each row (i.e. $\left[ \begin{array}{cccc} x_{11} & x_{12} & \ldots & x_{1D} \end{array} \right]$) represents a single data-point (like one MNIST image or one window of observations)

In lecture slides you will find the equations following the conventional mathematical approach, using column vectors, but you can easily map between column-major and row-major organisations using a matrix transpose.

***

## Linear and Affine Transforms

The basis of all linear models is the so called affine transform, which is a transform that implements a linear transformation and translation of the input features. The transforms we are going to use are parameterised by:

  * A weight matrix $\mathbf{W} \in \mathbb{R}^{D\times K}$: where element $w_{ik}$ is the weight from input $x_i$ to output $y_k$
  * A bias vector $\mathbf{b}\in R^{K}$ : where element $b_{k}$ is the bias for output $k$

Note, the bias is simply some additive term, and can be easily incorporated into an additional row in weight matrix and an additional input in the inputs which is set to $1.0$ (as in the below picture taken from the lecture slides). However, here (and in the code) we will keep them separate.

![Making Predictions](res/singleLayerNetWts-1.png)

For instance, for the above example of 5-dimensional input vector by $\mathbf{x} = (x_1, x_2, x_3, x_4, x_5)$, weight matrix $\mathbf{W}=\left[ \begin{array}{ccc}
w_{11} & w_{12} & w_{13} \\
w_{21} & w_{22} & w_{23} \\
w_{31} & w_{32} & w_{33} \\
w_{41} & w_{42} & w_{43} \\
w_{51} & x_{52} & 2_{53} \\ \end{array} \right]$, bias vector $\mathbf{b} = (b_1, b_2, b_3)$ and outputs $\mathbf{y} = (y_1, y_2, y_3)$, one can write the transformation as follows:

(for the $i$-th output)

(1) $
\begin{equation}
   y_i = b_i + \sum_j x_jw_{ji}
\end{equation}
$

or the equivalent vector form (where $\mathbf w_i$ is the $i$-th column of $\mathbf W$):

(2) $
\begin{equation}
   y_i = b_i + \mathbf x \mathbf w_i^T
\end{equation}
$

The same operation can be also written in matrix form, to compute all the outputs $\mathbf{y}$ at the same time:

(3) $
\begin{equation}
  \mathbf y=\mathbf x\mathbf W + \mathbf b
\end{equation}
$

This is equivalent to slides 12/13 in lecture 1, except we are using row vectors.

When $\mathbf{x}$ is a mini-batch (contains $B$ data-points of dimension $D$ each), i.e. $\left[ \begin{array}{cccc}
x_{11} & x_{12} & \ldots & x_{1D} \\
x_{21} & x_{22} & \ldots & x_{2D} \\
\cdots \\
x_{B1} & x_{B2} & \ldots & x_{BD} \\ \end{array} \right]$ equation (3) effectively becomes to be

(4) $
\begin{equation}
  \mathbf Y=\mathbf X\mathbf W + \mathbf b
\end{equation}
$

where both $\mathbf{X}\in\mathbb{R}^{B\times D}$ and $\mathbf{Y}\in\mathbb{R}^{B\times K}$ are matrices, and $\mathbf{b}$ needs to be <a href="http://docs.scipy.org/doc/numpy/user/basics.broadcasting.html">broadcasted</a> $B$ times (numpy will do this by default). However, we will not make an explicit distinction between a special case for $B=1$ and $B>1$ and simply use equation (3) instead, although $\mathbf{x}$ and hence $\mathbf{y}$ could be matrices. From an implementation point of view, it does not matter.

The desired functionality for matrix multiplication in numpy is provided by <a href="http://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html">numpy.dot</a> function. If you haven't use it so far, get familiar with it as we will use it extensively.

### A general note on random number generators

It is generally a good practice (for machine learning applications **not** for cryptography!) to seed a pseudo-random number generator once at the beginning of the experiment, and use it later through the code where necesarry. This makes it easier to reproduce results since random initialisations can be replicated. As such, within this course we are going use a single random generator object, similar to the below:

In [ ]:
import numpy

#initialise the random generator to be used later
seed=[2015, 10, 1]
random_generator = numpy.random.RandomState(seed)

## Exercise 1 

Using `numpy.dot`, implement **forward** propagation through the linear transform defined by equations (3) and (4) for $B=1$ and $B>1$. As data ($\mathbf{x}$) use `MNISTDataProvider` introduced last week. For the case when $B=1$, write a function to compute the 1st output ($y_1$) using equations (1) and (2). Check if the output is the same as the corresponding one obtained with numpy. 

Tip: To generate random data you can use `random_generator.uniform(-0.1, 0.1, (D, 10))` from above.

In [ ]:
from mlp.dataset import MNISTDataProvider

mnist_dp = MNISTDataProvider(dset='valid', batch_size=3, max_num_batches=1, randomize=False)

irange = 0.1
W = random_generator.uniform(-irange, irange, (784,10)) 
b = numpy.zeros((10,))


In [ ]:

mnist_dp.reset()

#implement following functions, then run the cell
def y1_equation_1(x, W, b):
    raise NotImplementedError()
    
def y1_equation_2(x, W, b):
    raise NotImplementedError()

def y_equation_3(x, W, b):
    #use numpy.dot
    raise NotImplementedError()

def y_equation_4(x, W, b):
    #use numpy.dot
    raise NotImplementedError()

for x, t in mnist_dp:
    y1e1 = y1_equation_1(x[0], W, b)
    y1e2 = y1_equation_2(x[0], W, b)
    ye3 = y_equation_3(x, W, b)
    ye4 = y_equation_4(x, W, b)

print 'y1e1', y1e1
print 'y1e1', y1e1
print 'ye3', ye3
print 'ye4', ye4
    

## Exercise 2

Modify the examples from Exercise 1 to perform **backward** propagation, that is, given $\mathbf{y}$ (obtained in the previous step) and weight matrix $\mathbf{W}$, project $\mathbf{y}$ onto the input space $\mathbf{x}$ (ignore or set to zero the biases towards $\mathbf{x}$ in backward pass). Mathematically, we are interested in the following transformation: $\mathbf{z}=\mathbf{y}\mathbf{W}^T$

***
## Exercise 3 (optional)

In case you do not fully understand how matrix-vector and/or matrix-matrix products work, consider implementing `my_dot_mat_mat` function  (you have been given `my_dot_vec_mat` code to look at as an example) which takes as the input the following arguments:

* D-dimensional input vector $\mathbf{x} = (x_1, x_2, \ldots, x_D) $.
* Weight matrix $\mathbf{W}\in\mathbb{R}^{D\times K}$:

and returns:

* K-dimensional output vector $\mathbf{y} = (y_1, \ldots, y_K) $

Your job is to write a variant that works in a mini-batch mode where both $\mathbf{x}\in\mathbb{R}^{B\times D}$ and $\mathbf{y}\in\mathbb{R}^{B\times K}$ are matrices in which each rows contain one of $B$ data-points from mini-batch (rather than  $\mathbf{x}\in\mathbb{R}^{D}$ and $\mathbf{y}\in\mathbb{R}^{K}$).

In [ ]:
def my_dot_vec_mat(x, W):
    J = x.shape[0]
    K = W.shape[1]
    assert (J == W.shape[0]), (
        "Number of columns of x expected to "
        " to be equal to the number of rows in "
        "W, bot got shapes %s, %s" % (x.shape, W.shape)
    )
    y = numpy.zeros((K,))
    for k in xrange(0, K):
        for j in xrange(0, J):
            y[k] += x[j] * W[j,k]
                
    return y

In [ ]:
irange = 0.1 #+-range from which we draw the random numbers

x = random_generator.uniform(-irange, irange, (5,)) 
W = random_generator.uniform(-irange, irange, (5,3)) 

y_my = my_dot_vec_mat(x, W)
y_np = numpy.dot(x, W)

same = numpy.allclose(y_my, y_np)

if same:
    print 'Well done!'
else:
    print 'Matrices are different:'
    print 'y_my is: ', y_my
    print 'y_np is: ', y_np

In [ ]:
def my_dot_mat_mat(x, W):
    I = x.shape[0]
    J = x.shape[1]
    K = W.shape[1]
    assert (J == W.shape[0]), (
        "Number of columns in of x expected to "
        " to be the same as rows in W, got"
    )
    #allocate the output container
    y = numpy.zeros((I, K))
    
    #implement here matrix-matrix inner product here
    raise NotImplementedError('Write me!')
                
    return y

Test whether you get comparable numbers to what numpy is producing:

In [ ]:
irange = 0.1 #+-range from which we draw the random numbers

x = random_generator.uniform(-irange, irange, (2,5)) 
W = random_generator.uniform(-irange, irange, (5,3)) 

y_my = my_dot_mat_mat(x, W)
y_np = numpy.dot(x, W)

same = numpy.allclose(y_my, y_np)

if same:
    print 'Well done!'
else:
    print 'Matrices are different:'
    print 'y_my is: ', y_my
    print 'y_np is: ', y_np

Now we benchmark each approach (we do it in separate cells, as timeit currently can measure whole cell execuiton only).

In [ ]:
#generate bit bigger matrices, to better evaluate timings
x = random_generator.uniform(-irange, irange, (10, 1000))
W = random_generator.uniform(-irange, irange, (1000, 100))

In [ ]:
print 'my_dot timings:'
%timeit -n10 my_dot_mat_mat(x, W)

In [ ]:
print 'numpy.dot timings:'
%timeit -n10 numpy.dot(x, W)

**Optional section ends here**
***

# Iterative learning of linear models

We will learn the model with stochastic gradient descent on N data-points using mean square error (MSE) loss function, which is defined as follows:

(5) $
E = \frac{1}{2} \sum_{n=1}^N ||\mathbf{y}^n - \mathbf{t}^n||^2 =  \sum_{n=1}^N E^n \\
  E^n = \frac{1}{2} ||\mathbf{y}^n - \mathbf{t}^n||^2
$

(6) $ E^n = \frac{1}{2} \sum_{k=1}^K (y_k^n - t_k^n)^2 $
  
Hence, the gradient w.r.t (with respect to) the $r$ output y of the model is defined as, so called delta function, $\delta_r$: 

(8) $\frac{\partial{E^n}}{\partial{y_{r}}} = (y^n_r - t^n_r) =  \delta^n_r \quad ; \quad
    \delta^n_r = y^n_r - t^n_r \\
    \frac{\partial{E}}{\partial{y_{r}}} = \sum_{n=1}^N \frac{\partial{E^n}}{\partial{y_{r}}} = \sum_{n=1}^N \delta^n_r
$

Similarly, using the above $\delta^n_r$ one can express the gradient of the  weight $w_{sr}$ (from the s-th input to the r-th output) for linear model and MSE cost as follows:

(9) $
    \frac{\partial{E^n}}{\partial{w_{sr}}} = (y^n_r - t^n_r)x_s^n =  \delta^n_r x_s^n \quad\\
    \frac{\partial{E}}{\partial{w_{sr}}} = \sum_{n=1}^N \frac{\partial{E^n}}{\partial{w_{rs}}} = \sum_{n=1}^N \delta^n_r x_s^n
$

and the gradient for bias parameter at the $r$-th output is:

(10) $
    \frac{\partial{E}}{\partial{b_{r}}} = \sum_{n=1}^N \frac{\partial{E^n}}{\partial{b_{r}}} = \sum_{n=1}^N \delta^n_r
$


![Making Predictions](res/singleLayerNetPredict.png)
 
  * Input vector $\mathbf{x} = (x_1, x_2, \ldots, x_D) $
  * Output scalar $y_1$
  * Weight matrix $\mathbf{W}$: $w_{ik}$ is the weight from input $x_i$ to output $y_k$. Note, here this is really a vector since a single scalar output, y_1.
  * Scalar bias $b$ for the only output in our model 
  * Scalar target $t$ for the only output in out model
  
First, ensure you can make use of the data provider (note, for training data has been normalised to zero mean and unit variance, hence different effective range than one can find in file):

In [ ]:
from mlp.dataset import MetOfficeDataProvider

modp = MetOfficeDataProvider(10, batch_size=10, max_num_batches=2, randomize=False)

%precision 2
for x, t in modp:
    print 'Observations: ', x
    print 'To predict: ', t

## Exercise 4

The below code implements a very simple variant of stochastic gradient descent for the rainfall prediction example. Your task is to implement 5 functions in the next cell and then run two next cells that 1) build sgd functions and 2) run the actual training.

In [ ]:

#When implementing those, take into account the mini-batch case, for which one is
#expected to sum the errors for each example

def fprop(x, W, b):
    #code implementing eq. (3)
    raise NotImplementedError('Write me!')

def cost(y, t):
    #Mean Square Error cost, equation (5)
    raise NotImplementedError('Write me!')

def cost_grad(y, t):
    #Gradient of the cost w.r.t y equation (8)
    raise NotImplementedError('Write me!')

def cost_wrt_W(cost_grad, x):
    #Gradient of the cost w.r.t W, equation (9)
    raise NotImplementedError('Write me!')
    
def cost_wrt_b(cost_grad):
    #Gradient of the cost w.r.t to b, equation (10)
    raise NotImplementedError('Write me!')


In [ ]:

def sgd_epoch(data_provider, W, b, learning_rate):
    mse_stats = []
    
    #get the minibatch of data
    for x, t in data_provider:
        
        #1. get the estimate of y
        y = fprop(x, W, b)

        #2. compute the loss function
        tmp = cost(y, t)
        mse_stats.append(tmp)
        
        #3. compute the grad of the cost w.r.t the output layer activation y
        #i.e. how the cost changes when output y changes
        cost_grad_deltas = cost_grad(y, t)

        #4. compute the gradients w.r.t model's parameters
        grad_W = cost_wrt_W(cost_grad_deltas, x)
        grad_b = cost_wrt_b(cost_grad_deltas)

        #4. Update the model, we update with the mean gradient
        # over the minibatch, rather than sum of particular gradients
        # in a minibatch, to do so we scale the learning rate by batch_size
        batch_size = x.shape[0]
        effect_learn_rate = learning_rate / batch_size

        W = W - effect_learn_rate * grad_W
        b = b - effect_learn_rate * grad_b
    
    return W, b, numpy.mean(mse_stats)

def sgd(data_provider, W, b, learning_rate=0.1, max_epochs=10):
    
    for epoch in xrange(0, max_epochs):
        #reset the data provider
        data_provider.reset()
        
        #train for one epoch
        W, b, mean_cost = \
            sgd_epoch(data_provider, W, b, learning_rate)
                
        print "MSE training cost after %d-th epoch is %f" % (epoch + 1, mean_cost)
    
    return W, b
        
        

In [ ]:

#some hyper-parameters
window_size = 12
irange = 0.1
learning_rate = 0.01
max_epochs=40

# note, while developing you can set max_num_batches to some positive number to limit
# the number of training data-points (you will get feedback faster)
mdp = MetOfficeDataProvider(window_size, batch_size=10, max_num_batches=-100, randomize=False)

#initialise the parameters
W = random_generator.uniform(-irange, irange, (window_size, 1))
b = random_generator.uniform(-irange, irange, (1, ))

#train the model
sgd(mdp, W, b, learning_rate=learning_rate, max_epochs=max_epochs)


## Exercise 5

Modify the above prediction (regression) problem so the model makes a binary classification whether the the weather is going to be one of those \{rainy, not-rainy} (look at slide 12 of the 2nd lecture)

Tip: You need to introduce the following changes:
1. Modify `MetOfficeDataProvider` (for example, inherit from MetOfficeDataProvider to create a new class MetOfficeDataProviderBin) and modify `next()` function so it returns as `targets` either 0 (not-rainy - if the the amount of rain [before mean/variance normalisation] is equal to 0) or 1 (rainy -- otherwise).
2. Modify the functions from previous exercise so the fprop implements `sigmoid` on top of affine transform.
3. Modify cost function to binary cross-entropy
4. Make sure you compute the gradients correctly (as you have changed both the output and the cost)
